In [1]:
import sys
sys.path.append('C:/Users/wangl/Desktop/root/RflysimUT_Django')
from RflysimUT import settings
from asgiref.sync import sync_to_async

In [2]:
from tool import *
from cache.write_to_redis import refresh_redis
from cache.write_to_redis import write_to_redis
from cache.redis_db import redis_db as rdb
from cache.config import *
import django

In [3]:
# 初始化django，连接到redis
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "RflysimUT.settings")
django.setup()
cdb_client = rdb(host=G_REDIS_HOST, port=G_REDIS_PORT, db=G_REDIS_DB, password=G_REDIS_PW) # 存储数据库数据的redis库
drone_db = rdb(host=G_REDIS_HOST, port=G_REDIS_PORT, db=G_REDIS_DB + 1, password=G_REDIS_PW) # 存储无人机实时信息的redis库
# 清空redis残余数据
cdb_client.rdb.flushall()
drone_db.rdb.flushall()

# 将初始数据写入redis
write_to_redis = sync_to_async(write_to_redis)
async def write_to_redis_caller():
    result = await write_to_redis(cdb_client)
    return result
await write_to_redis_caller()

# 定义仿真方案的id
proposal_id = '1'
# 创建航路网
[net, nodes_list, edges_dict] = create_net(cdb_client, proposal_id)
# 定义flight_requirements 、 network_controls 以及标记管控是否已经作用过的flag
flight_requirements = []
network_controls = {}
network_controls_flag = {}
# 获取无人机数据
drones = get_drones(cdb_client, proposal_id)
drones_id = drones.keys()
# 定义无人机的起始创建位置、终点位置、当前任务的出发位置
initial_positions = {}
final_positions ={}
start_positions = {}
# 定义字典，用于存储每个无人机当前任务的起飞、降落机场
takeoff_ports = {}
land_ports = {}
# 获取每个机场停放无人机的停放点位置
places = cdb_client.get_data(f'{proposal_id}:place:1')
# 每个机场定义一个队列，用于给无人机随机分配停放位置
que = dict.fromkeys(places.keys(), None)
for key in que:
    que[key] = [x for x in range(len(places[key]))]
# 定义已完成飞行任务的无人机队列，用于自动生成新的飞行任务
finish_que = []

Data has been written to redis


In [4]:
import time
import airsim
import math
from cache.config import IP
import numpy as np


# 连接到AirSim模拟器
# client_pip = airsim.MultirotorClient()
client_pip = airsim.MultirotorClient(IP)
client_pip.confirmConnection()


def pip_cre_with_color(node1, node2, node1_name, node2_name, color):
    # 计算节点之间的距离
    length = math.sqrt((node2[0] - node1[0])**2 + (node2[1] - node1[1])**2)

    # 设置飞行路径的尺度
    pip_scale = airsim.Vector3r(length-20*math.sqrt(3), 15, 20)

    # 计算飞行路径的中心点
    pip_cen = [(node2[0] + node1[0]) / 2, (node2[1] + node1[1]) / 2, node1[2]]  # 高度保持不变

    # 计算方向向量并转换为四元数
    pip_dir = [node2[0] - node1[0], node2[1] - node1[1], 0]
    pip_orientation = vector_to_quaternion(pip_dir)

    # 设置飞行路径的姿态
    pip_pose = airsim.Pose(airsim.Vector3r(pip_cen[0], pip_cen[1], pip_cen[2]),
                           pip_orientation)

    # 添加飞行路径到AirSim模拟器
    client_pip.addFlightPip(f'{node1_name}to{node2_name}', pip_pose, pip_scale, color, cube=True)
    print(f'{node1_name}to{node2_name}')


def vector_to_quaternion(v):
    """将方向向量转换为四元数"""
    # 归一化方向向量
    v_mag = math.sqrt(v[0] ** 2 + v[1] ** 2 + v[2] ** 2)
    if v_mag == 0:
        return airsim.Quaternionr(0, 0, 0, 1)  # 如果向量为零向量，返回单位四元数

    v_normalized = [v[0] / v_mag, v[1] / v_mag, v[2] / v_mag]

    # 计算与Z轴的夹角
    yaw = math.atan2(v_normalized[1], v_normalized[0])

    # 将偏航角转换为四元数
    qw = math.cos(yaw / 2)
    qz = math.sin(yaw / 2)

    return airsim.Quaternionr(0, 0, qz, qw)


def port_cre(node):
    """创建机场"""
    port_pose = airsim.Pose(airsim.Vector3r(node[0],node[1],-60),airsim.Quaternionr(0,0,0,1))

    port_scale = airsim.Vector3r(50,50,120)

    port_color = [0.4, 0.843, 0.608, 0.8]

    client_pip.addFlightPip(f"{node}", port_pose, port_scale, port_color, cube=False)


def dynamic_change_pipe(pip_name):
    """管控航路管道颜色更改"""
    node_names = pip_name.split('to')

    nodes = []

    color2 = [1, 0, 0, 0.5]

    # 遍历分割后的字符串列表
    for node_name in node_names:
        # 去掉方括号并分割字符串以获取坐标值
        coords = node_name.strip('[]').split(',')
        # 将坐标值转换为整数列表
        node = list(map(lambda x: int(float(x)), coords))
        # 将转换后的点添加到点列表中
        nodes.append(node)

    client_pip.delFlightPip(pip_name)
    pip_cre_with_color(node1=nodes[0], node2=nodes[1], node1_name=node_names[1], node2_name=node_names[0], color=color2)


# 添加航路管道
color1 = [0.29, 0.729, 1, 0.65]
nodes_dict = dict.fromkeys(nodes_list)
pip_flag = {}
for edge in edges_dict:
    [start_node_id, end_node_id] = edges_dict[edge]
    [start_node, end_node] = [net[start_node_id][end_node_id][1][0], net[start_node_id][end_node_id][1][-1]]
    if f'{end_node}to{start_node}' in pip_flag:
        continue
    pip_cre_with_color(start_node, end_node, f'{start_node}', f'{end_node}', color1)
    pip_flag[f'{start_node}to{end_node}'] = 1
    nodes_dict[start_node_id] = start_node
    nodes_dict[end_node_id] = end_node
    time.sleep(0.1)

# 添加圆柱形节点
cir_scale = airsim.Vector3r(40, 40, 20)
cir_color = [0.976, 0.973, 0.422, 0.65]
for node_id in nodes_list:
    # 生成每个节点的名称，如 'circle1', 'circle2', 等
    circle_name = f"circle{node_id}"
    # 设置节点的位置
    cir_pose = airsim.Pose(airsim.Vector3r(nodes_dict[node_id][0], nodes_dict[node_id][1], nodes_dict[node_id][2]),
                           airsim.Quaternionr(0, 0, 0, 1))

    # 调用 addFlightPip 函数
    client_pip.addFlightPip(circle_name, cir_pose, cir_scale, cir_color, cube=False)
    time.sleep(0.1)


# 以二维向量的形式定义机场的坐标
ports_pos = {
    '(1,1)': np.array([1, 1]),
    '(23,328)': np.array([23, 328]), 
    '(231,211)': np.array([231, 211]),
    '(190,-7)': np.array([190, -7]), 
    '(-46,198)': np.array([-46, 198]), 
    '(275,-62)': np.array([275, -62]), 
    '(196,-80)': np.array([196, -80]),
    '(88,-101)': np.array([88, -101]), 
    '(105,142)': np.array([105, 142]),  
    '(294,182)': np.array([294, 182]),  
    '(262,86)': np.array([262, 86]),
    '(-165,-122)': np.array([-165, -122]),
    '(-81,-118)': np.array([-81, -118])
}

# 在UE场景中添加机场模型
loc = airsim.Vector3r(1, 1, 0)
client_pip.addLandingPort('(1,1)', loc, 7)

loc = airsim.Vector3r(23, 328, 0)
client_pip.addLandingPort('(23,328)', loc, 7)

loc = airsim.Vector3r(231, 211, 0)
client_pip.addLandingPort('(231,211)', loc, 7)

loc = airsim.Vector3r(190, -7, 0)
client_pip.addLandingPort('(190,-7)', loc, 7)

loc = airsim.Vector3r(-46, 198, 0)
client_pip.addLandingPort('(-46,198)', loc, 7)

loc = airsim.Vector3r(275, -62, 0)
client_pip.addLandingPort('(275,-62)', loc, 10)

loc = airsim.Vector3r(196, -80, 0)
client_pip.addLandingPort('(196,-80)', loc, 10)

loc = airsim.Vector3r(88, -101, 0)
client_pip.addLandingPort('(88,-101)', loc, 10)

loc = airsim.Vector3r(105, 142, 0)
client_pip.addLandingPort('(105,142)', loc, 10)

loc = airsim.Vector3r(294, 182, 0)
client_pip.addLandingPort('(294,182)', loc, 10)

loc = airsim.Vector3r(262, 86, 0)
client_pip.addLandingPort('(262,86)', loc, 10)

loc = airsim.Vector3r(-165, -122, 0)
client_pip.addLandingPort('(-165,-122)', loc, 10)

loc = airsim.Vector3r(-81, -118, 0)
client_pip.addLandingPort('(-81,-118)', loc, 1)

# 在机场所在位置添加圆柱形管道
port_cre([1, 1, -120])
port_cre([23, 328, -120])
port_cre([231, 211, -120])
port_cre([190, -7, -120])
port_cre([-46, 198, -120])
port_cre([275, -62, -120])
port_cre([196, -80, -120])
port_cre([88, -101, -120])
port_cre([105, 142, -120])
port_cre([294, 182, -120])
port_cre([262, 86, -120])
port_cre([-165, -122, -120])
port_cre([-81, -118, -120])


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

[-251.0, 162.0, -120.0]to[-246.0, 124.0, -120.0]
[37.0, -109.0, -120.0]to[0.0, -111.0, -120.0]
[-172.0, 105.0, -120.0]to[-120.0, 245.0, -120.0]
[-256.0, 349.0, -120.0]to[-170.0, 472.0, -120.0]
[1.0, 1.0, -120.0]to[80.0, -5.0, -120.0]
[-189.0, 297.0, -120.0]to[-120.0, 245.0, -120.0]
[116.0, -96.0, -120.0]to[88.0, -101.0, -120.0]
[105.0, 142.0, -120.0]to[80.0, -5.0, -120.0]
[0.0, -111.0, -120.0]to[1.0, 1.0, -120.0]
[294.0, 182.0, -120.0]to[262.0, 86.0, -120.0]
[-120.0, 245.0, -120.0]to[-46.0, 198.0, -120.0]
[-256.0, 349.0, -120.0]to[-189.0, 297.0, -120.0]
[-242.0, 84.0, -120.0]to[-164.0, 65.0, -120.0]
[158.0, 247.0, -120.0]to[89.0, 287.0, -120.0]
[-246.0, 124.0, -120.0]to[-321.0, 151.0, -120.0]
[231.0, 211.0, -120.0]to[294.0, 182.0, -120.0]
[-106.0, 420.0, -120.0]to[-189.0, 297.0, -120.0]
[-246.0, 124.0, -120.0]to[-172.0, 105.0, -120.0]
[-164.0, 65.0, -120.0]to[-160.0, 17.0, -120.0]
[-165.0, -122.0, -120.0]to[-254.0, -119.0

In [5]:
def listen():
    global flight_requirements, net, proposal_id, nodes_list, network_controls, edges_dict, nodes_dict, drones, \
        drones_id, cdb_client, drone_db, takeoff_ports, land_ports
    while True:
        refresh_redis(cdb_client)
        # refresh drone data
        drones_temp = get_drones(cdb_client, proposal_id)
        drones_id = drones_temp.keys()

        # from requirement to plan （根据requirement进行路径规划，如需自定义路径规划可以更改djstra函数，或者将此部分代码注释掉）
        flight_requirements_new = get_flight_requirements(cdb_client, proposal_id)
        if not len(flight_requirements_new) == len(flight_requirements):
            for new_req in flight_requirements_new:
                if new_req not in flight_requirements:
                    [points, edges, nodes] = djstra(net, nodes_list, new_req[2], new_req[3])
                    new_req[4]['flight_path'] = points
                    new_req[4]['airway_path'] = edges
                    takeoff_ports[new_req[1]] = new_req[4]['start_airport_name']
                    land_ports[new_req[1]] = new_req[4]['end_airport_name']
                    drones[new_req[1]] = [points, edges, nodes, new_req[0]]
                    print(new_req[1])
                    print(drones[new_req[1]])
                    cdb_client.set_data(f'{proposal_id}:flight_plan:{new_req[0]}', new_req[4], None)
        flight_requirements = flight_requirements_new

        # refresh network control
        network_controls = get_network_control(cdb_client, proposal_id)
        for ctrl in network_controls:
            if network_controls[ctrl] == 0:
                del network_controls[ctrl]
            elif (not network_controls_flag.get(ctrl, False)) or \
                    (network_controls[ctrl] != network_controls_flag[ctrl]):
                if network_controls[ctrl] == 1:
                    [start_node_id, end_node_id] = edges_dict[ctrl]
                    dynamic_change_pipe(f'{nodes_dict[start_node_id]}to{nodes_dict[end_node_id]}')
                    network_controls_flag[ctrl] = 1

        # from network control to plan （结合管控信息进行路径规划，如需自定义路径规划可以更改djstra函数，或者将此部分代码注释掉）
        for drone in drones:
            flag_ctrl = 0
            if len(drones[drone][1]) < 2:
                continue
            for edge in drones[drone][1][1:]:
                if network_controls.get(edge, False) and network_controls[edge] == 1:
                    flag_ctrl = 1
            if flag_ctrl == 1:
                print(f'drone_{drone} needs to plan a new path due to network control')
                drones[drone] = plan_a_new_path(net, nodes_list, drones[drone][2][0], drones[drone][2][-1],
                                                network_controls, edges_dict)
        time.sleep(1)

In [ ]:
from parameter import *
import threading

def mysat(val, max_val):
    norm_val = np.linalg.norm(val)
    if norm_val > max_val:
        return val * max_val / norm_val
    return val


drones_pos = {}
# create client
clients = {}


def run(client, drone, num):
    global drones, drones_id, drones_pos
    flag_first = True
    client.pickBox(f'drone_{drone}', takeoff_ports[drone])
    time.sleep(10)
    client.moveToPositionAsync(start_positions[drone].x_val-initial_positions[drone].x_val,
                               start_positions[drone].y_val-initial_positions[drone].y_val, -120, 10,
                               vehicle_name=f'drone_{drone}')
    if drones[drone]:
        drones[drone][0].pop(0)
        drones[drone][2].pop(0)
        velocity = np.array([0.0, 0.0, 0.0])
        while drones[drone][0]:
            # 更新无人机当前位置
            position = clients[drone].getMultirotorState(
                vehicle_name=f'drone_{drone}').kinematics_estimated.position
            while flag_first:
                # 确保无人机升空到指定高度
                position = clients[drone].getMultirotorState(
                    vehicle_name=f'drone_{drone}').kinematics_estimated.position
                if position.z_val < -110:
                    flag_first = False
                    que[takeoff_ports[drone]].append(num)
                    break
                time.sleep(1)
            drones_pos[drone] = [position.x_val + initial_positions[drone].x_val,
                                 position.y_val + initial_positions[drone].y_val,
                                 position.z_val + initial_positions[drone].z_val]
            position = np.array(drones_pos[drone])
            target = np.array(drones[drone][0][0])

            # 吸引力方向：无人机与当前目标点的向量
            el = target - position
            el_norm = np.linalg.norm(el)
            if el_norm != 0:
                attract_force = K_attract * (el / el_norm)  # 吸引力与目标点距离成正比
            else:
                attract_force = np.array([0.0, 0.0, 0.0])

            # 计算与其他无人机的斥力
            repulsion = 0
            other_drones = drones_pos.copy()
            for other_drone in other_drones:
                if drone == other_drone:
                    continue
                if other_drone in drones_pos:
                    direction = position - np.array(drones_pos[other_drone])
                distance = np.linalg.norm(direction)
                if 0 < distance < rs:
                    repulsion_magnitude = K_repulse * (rs - distance) ** 3 / (distance ** 2)
                    repulsion += repulsion_magnitude * (direction / distance)
                elif rs <= distance < ra:  # 增加一个平滑过渡的区域
                    repulsion_magnitude = 1 * K_repulse * (ra - distance) / (ra - rs)
                    repulsion += repulsion_magnitude * (direction / distance)

            # 合力
            total_force = attract_force + repulsion
            # 平滑速度更新 (插值)
            velocity = 0.5 * velocity + 0.5 * mysat(total_force, vmax)
            vx, vy, vz = velocity

            clients[drone].moveByVelocityAsync(vx, vy, vz, time_step*1.5, vehicle_name=f'drone_{drone}')

            # 检查是否到达当前目标点
            distance_to_target = np.linalg.norm(target - position)
            if distance_to_target < 15 and len(drones[drone][0]) > 0:
                drones[drone][0].pop(0)
                drones[drone][1].pop(0)
                drones[drone][2].pop(0)
                if len(drones[drone][0]) == 0:
                    while not que[land_ports[drone]]:
                        time.sleep(1)
                    num = que[land_ports[drone]].pop(0)
                    final_positions[drone] = airsim.Vector3r(places[land_ports[drone]][num][0],
                                                             places[land_ports[drone]][num][1],
                                                             places[land_ports[drone]][num][2])
                    client.moveToPositionAsync(final_positions[drone].x_val-initial_positions[drone].x_val,
                                               final_positions[drone].y_val-initial_positions[drone].y_val, -5, 10,
                                               vehicle_name=f'drone_{drone}')
                    time.sleep(20)
                    # 执行放货动作
                    client.dropBox(f'drone_{drone}', land_ports[drone])
                    time.sleep(5)
                    client.landAsync(vehicle_name = f'drone_{drone}')
                    time.sleep(5)
                    cdb_client.rdb.delete(f'{proposal_id}:drone:{drone}')
                    cdb_client.rdb.delete(f'{proposal_id}:flight_plan:{drones[drone][3]}')
                    finish_que.append([drone, land_ports[drone]])
                    del drones_pos[drone]
                    del drones[drone]
                    del takeoff_ports[drone]
                    que[land_ports[drone]].append(num)
                    del land_ports[drone]
                    drone_db.rdb.delete(f'{proposal_id}:drone_current:{drone}')
                    break

            time.sleep(time_step)


def get_drone_pos():
    global drones_pos, drones_id, drone_db, proposal_id, drones
    while True:
        drones_pos_temp = drones_pos.copy()
        for drone in drones_pos_temp:
            if drone not in drones_pos or drone not in drones:
                continue
            if drones_pos[drone] and drones[drone][1]:
                refresh_drone(drone_db, proposal_id, drone,
                              [initial_positions[drone].x_val, initial_positions[drone].y_val, initial_positions[drone].z_val],
                              drones_pos[drone], drones[drone][1][0])
            else:
                refresh_drone(drone_db, proposal_id, drone,
                              [initial_positions[drone].x_val, initial_positions[drone].y_val, initial_positions[drone].z_val],
                              drones_pos[drone], -1)
        time.sleep(1)


def AddDrones():
    global land_ports, ports_pos, finish_que, takeoff_ports
    from source.flight_plan.models import FlightRequirements
    from source.proposal.models import Proposals
    from source.uav.models import Drones
    from source.air_road.models import Airports
    import datetime

    proposal = Proposals.objects.get(name='proposal')

    while True:
        ports_count = dict.fromkeys(ports_pos.keys(), 0)
        land_ports_copy = land_ports.copy()
        for land_drone in land_ports_copy:
            ports_count[land_ports_copy[land_drone]] += 1
        if finish_que:
            [finish_drone, finish_port] = finish_que.pop(0)
            port_flag = False
            while not port_flag:
                for port in ports_count:
                    if ports_count[port] < 10 and port!=finish_port:
                        land_ports[finish_drone] = port
                        port_flag = True
                        break
            takeoff_ports[finish_drone] = finish_port
            current_time = datetime.datetime.now()
            drone_sample = Drones.objects.get(name=f'drone_{finish_drone}')
            takeoff_airport = Airports.objects.get(name=finish_port)
            land_airport = Airports.objects.get(name=land_ports[finish_drone])
            flight_req_sample = FlightRequirements(proposal=proposal, name=f'drone_{finish_drone}_{current_time}',
                                                   drone=drone_sample, priority=1, start_airport=takeoff_airport,
                                                   end_airport=land_airport, type='0', end_time=current_time)
            flight_req_sample.save()
            print(f'new requirement has been created for drone_{finish_drone}')
            finish_num = que[takeoff_ports[finish_drone]].pop(0)
            start_positions[finish_drone] = airsim.Vector3r(places[takeoff_ports[finish_drone]][finish_num][0],
                                                       places[takeoff_ports[finish_drone]][finish_num][1],
                                                       places[takeoff_ports[finish_drone]][finish_num][2])
            clients[finish_drone].takeoffAsync(vehicle_name=f'drone_{finish_drone}')
            time.sleep(5)
            t_rerun = threading.Thread(target=run, args=[clients[finish_drone], finish_drone, finish_num])
            t_rerun.start()


if __name__ == "__main__":
    t_listen = threading.Thread(target=listen)
    t_listen.start()
    t_adddrone = threading.Thread(target=AddDrones)
    t_adddrone.start()
    t_pos = threading.Thread(target=get_drone_pos)
    t_pos.start()

    while True:
        try:
            drones_id_temp = drones.copy()
            for drone in drones_id_temp:
                if drones[drone] and (drone not in clients):
                    clients[drone] = airsim.MultirotorClient(IP)
                    clients[drone].confirmConnection()
                    num = que[takeoff_ports[drone]].pop(0)
                    initial_positions[drone] = airsim.Vector3r(places[takeoff_ports[drone]][num][0],
                                                               places[takeoff_ports[drone]][num][1],
                                                               places[takeoff_ports[drone]][num][2])
                    start_positions[drone] = initial_positions[drone]
                    init_pose = airsim.Pose(position_val=initial_positions[drone],
                                            orientation_val=airsim.Quaternionr(0, 0, 0, 1))
                    clients[drone].simAddVehicle(f'drone_{drone}', 'simpleflight', pose=init_pose)
                    clients[drone].enableApiControl(True, vehicle_name=f'drone_{drone}')
                    clients[drone].armDisarm(True, vehicle_name=f'drone_{drone}')
                    time.sleep(2)
                    clients[drone].takeoffAsync(vehicle_name=f'drone_{drone}')
                    time.sleep(5)
                    t_run = threading.Thread(target=run, args=[clients[drone], drone, num])
                    t_run.start()
            time.sleep(1)
        except Exception as e:
            print(e)

65
[[[294.0, 182.0, -120.0], [262.0, 86.0, -120.0], [275.0, -62.0, -120.0]], [36, 34], [25, 38, 36], 138]
9
[[[23.0, 328.0, -120.0], [89.0, 287.0, -120.0], [158.0, 247.0, -120.0], [231.0, 211.0, -120.0]], [61, 48, 42], [12, 31, 3, 22], 9]
86
[[[262.0, 86.0, -120.0], [275.0, -62.0, -120.0]], [34], [38, 36], 87]
39
[[[275.0, -62.0, -120.0], [196.0, -80.0, -120.0]], [28], [36, 23], 40]
98
[[[-165.0, -122.0, -120.0], [-121.0, -122.0, -120.0], [-81.0, -118.0, -120.0]], [69, 68], [30, 7, 15], 99]
84
[[[275.0, -62.0, -120.0], [196.0, -80.0, -120.0]], [28], [36, 23], 154]
48
[[[196.0, -80.0, -120.0], [116.0, -96.0, -120.0], [88.0, -101.0, -120.0]], [24, 16], [23, 14, 18], 49]
90
[[[262.0, 86.0, -120.0], [275.0, -62.0, -120.0]], [34], [38, 36], 91]
80
[[[294.0, 182.0, -120.0], [262.0, 86.0, -120.0]], [36], [25, 38], 81]
1
[[[1.0, 1.0, -120.0], [29.0, 151.0, -120.0], [89.0, 287.0, -120.0], [23.0, 328.0, -120.0]], [1, 49, 62], [27, 44, 31, 12], 1]
37
[[[275.0, -62.0, -120.0], [196.0, -80.0, -120.

Exception in thread Thread-6 (listen):
Traceback (most recent call last):
  File "C:\Users\wangl\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\wangl\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-25fc9bf6b4d3>", line 11, in listen
  File "C:\Users\wangl\Desktop\root\RflysimUT_Django\demo\tool.py", line 42, in get_flight_requirements
    start_airport_id = value['start_airport']
TypeError: 'bool' object is not subscriptable
Exception in thread Thread-47 (run):
Traceback (most recent call last):
  File "C:\Users\wangl\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\wangl\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
Exception in thread     self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-ef7ad12

Request timed out


Exception in thread Thread-48 (run):
Traceback (most recent call last):
  File "C:\Users\wangl\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\wangl\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-ef7ad1213bb7>", line 30, in run
  File "C:\Users\wangl\Desktop\root\RflysimUT_Django\venv\lib\site-packages\airsim\client.py", line 1604, in getMultirotorState
  File "C:\Users\wangl\Desktop\root\RflysimUT_Django\venv\lib\site-packages\msgpackrpc\session.py", line 41, in call
    return self.send_request(method, args).get()
  File "C:\Users\wangl\Desktop\root\RflysimUT_Django\venv\lib\site-packages\msgpackrpc\future.py", line 43, in get
    raise self._error
msgpackrpc.error.TimeoutError: Request timed out


Retry connection over the limit
Retry connection over the limit
Retry connection over the limit
Retry connection over the limit
Retry connection over the limit
Retry connection over the limit
Retry connection over the limit
